# Local Deployment

In [1]:
# Importing Dependencies
import torch
import torch.nn.functional as F
from PIL import Image
import open_clip
import os
from typing import List
from pymongo import MongoClient

In [2]:
# Check if GPU is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Load model and tokenizer
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k', device=device)
tokenizer = open_clip.get_tokenizer('ViT-B-32')

In [4]:
# Create a MongoDB client
client = MongoClient('mongodb://localhost:27017')  # replace with your connection string

# Create a database and a collection within the database
db = client['image-embeddings']
embeddings_collection = db['scene-sense']

In [5]:
# Function to get embeddings from image and store in MongoDB
def process_image_dir(directory: str):
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(directory, filename)
            
            # Check if the path already exists in the database
            existing_document = embeddings_collection.find_one({'image_path': image_path})
            if existing_document is not None:
                # If the path already exists in the database, skip this image
                continue
                
            # Else, compute the embedding and store it in the database
            image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
            with torch.no_grad(), torch.cuda.amp.autocast():
                image_features = model.encode_image(image)
                image_features /= image_features.norm(dim=-1, keepdim=True)
            
            # Save to MongoDB
            document = {
                'image_path': image_path,
                'embedding': image_features.cpu().numpy().tolist()  # convert tensor to list for storage
            }
            embeddings_collection.insert_one(document)

In [6]:
# Search for similar images given a query in DB
def run_query(query: str, top_k: int = 5) -> List[str]:
    # Tokenize and encode the query
    text = tokenizer([query]).to(device)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    # Get all embeddings from the database
    documents = embeddings_collection.find()

    # Compute similarities with all image embeddings and get top k
    similarities = []
    for document in documents:
        image_path = document['image_path']
        image_features = torch.tensor(document['embedding'], device=device)
        similarity = F.cosine_similarity(text_features, image_features.unsqueeze(0))
        similarities.append((similarity.item(), image_path))

    # Get top k results
    top_results = sorted(similarities, key=lambda x: x[0], reverse=True)[:top_k]
    for sim, path in top_results:
        print(f"Similarity: {sim}, Image Path: {path}")

In [7]:
# Create Embeddings
process_image_dir('sample_images')

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 649da4180dbc637b2d51c044, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>

In [ ]:
# Search for similar images
run_query("dog")